In [ ]:
import requests
from bs4 import BeautifulSoup
import re
from time import sleep
import os
from urllib.request import urlretrieve

In [ ]:
new_list = []
url = 'https://money.udn.com/money/index'

## 1.填寫headers
headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36'}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')
#print(response.text)

## 2.抓取即時區域所有新聞title以及新聞內容連結link，並將每篇新聞的連結link append到new_list內
latest_story = soup.findAll('li', {'class':'latest story'})
story_content = latest_story[0].findAll('div', {'class':'story__content'})
#print(story_content)
#print("====")

for all_a_tags in story_content:
    a_tags = all_a_tags.findAll('a')
    for i in a_tags:
        print('title: {}, link: {}'.format(i.h3.text.strip(), 'https://money.udn.com'+i['href']))
        new_list.append('https://money.udn.com'+i['href'])

In [ ]:
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)

        
def save_news(url):
    ## 3.填寫headers
    headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36'}
    response = requests.get(url, headers=headers)
    sleep(1)
    
    soup = BeautifulSoup(response.text, 'html.parser')
    ## 4. 使用find_all去抓取每篇新聞的title(指定是哪個tag以及指定id)，將結果所產生之list放到new_title變數內
    
    new_title = soup.findAll('h1', {'id':'story_art_title'})
    
    save_path = './news/'+new_title[0].text.replace(':','').replace("?", "").strip()

    article_body = soup.find_all('section',{'id':'article_body'})
    print('new title: {}'.format(new_title[0].text))

    ## create folder
    createFolder(save_path)
    soup2 = BeautifulSoup(str(article_body[0]), 'html.parser')


    ## save article contens in folder
    with open(save_path+'/contents.txt', 'w', encoding='utf-8') as f:  
        ## 5. 抓取文章內p tags，並將新聞內文寫入contents.txt檔案內
        p_tags = soup2.findAll('p')
        print('new content: ')
        for i in p_tags:
            print(i.text)
            f.write(i.text)

    ## save image in article        
    
    ## 6. 抓取文章內所有img tags，並將圖片下載並存入對應新聞資料夾下
    print('save image: ')
    img_tags = soup2.findAll('img')
    for i in img_tags:
        print(i['src'])
        urlretrieve(i['src'], save_path+'/'+ i['title'].replace(':','').replace("?", "").replace("/", "").replace("\\",'').strip() +'.jpg')
    print('==========\n\n')

In [ ]:
for new_url in new_list:
    print(new_url)
    save_news(new_url)

# 優化版本
- 針對最新的新聞頁面進行動態爬蟲

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
from time import sleep
import os
from urllib.request import urlretrieve

In [ ]:
url = 'https://money.udn.com/rank/ajax_newest/1001/0/{}'
headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36'}
new_list = []

# 要搜尋的頁數
for i in range(1,2):
    response3 = requests.get(url.format(i), headers=headers)
    soup4 = BeautifulSoup(response3.text, 'html.parser')
    new_title = soup4.findAll('div', {'class':'story__content'})

    for j in new_title:
        print("title: {}, link: {}".format(j.a["title"], j.a["href"]))
        new_list.append(j['href'])

In [ ]:
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)
        
def save_news(url):    
    ## 3.填寫headers
    headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36'}
    response = requests.get(url, headers=headers)
    sleep(1)
    
    soup = BeautifulSoup(response.text, 'html.parser')
    
    ## 4. 使用find_all去抓取每篇新聞的title(指定是哪個tag以及指定id)，將結果所產生之list放到new_title變數內
    new_title = soup.findAll('h1', {'id':'story_art_title'})
    save_path = './news/'+new_title[0].text.replace(':','').replace("?", "").replace("/", "").replace("\\",'').strip()
    article_body = soup.find_all('section',{'id':'article_body'})
    print('new title: {}'.format(new_title[0].text))

    ## create folder
    createFolder(save_path)
    soup2 = BeautifulSoup(str(article_body[0]), 'html.parser')

    ## save article contens in folder
    with open(save_path+'/contents.txt', 'w', encoding='utf-8') as f:  
        ## 5. 抓取文章內p tags，並將新聞內文寫入contents.txt檔案內
        p_tags = soup2.findAll('p')
        print('new content: ')
        for i in p_tags:
            print(i.text)
            f.write(i.text)

    ## save image in article        
    
    ## 6. 抓取文章內所有img tags，並將圖片下載並存入對應新聞資料夾下
    print('save image: ')
    img_tags = soup2.findAll('img')
    for i in img_tags:
        print(i['src'])
        urlretrieve(i['src'], save_path+'/'+ i['title'].replace(':','').replace("?", "").replace("/", "").replace("\\",'').strip() +'.jpg')
    print('==========\n\n')

In [ ]:
for new_url in new_list:
    print(new_url)
    save_news(new_url)